In [22]:
import matplotlib.pyplot as plt 
from sklearn import metrics
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns

import numpy as np # linear algebra
import pandas as pd #data processing

import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df = pd.read_csv('datasets/gossipcop_data_2.csv')
df

title label
0                 Trump blocks Chrissy Teigen on Twitter  REAL
1        Lady Gaga Swigs Vinegar Diet For Bradley Cooper  FAKE
2      Kim Cattrall pleads with fans to help find mis...  REAL
3          Jimmy Kimmel Mocks Kanye West's TMZ Interview  REAL
4      Kashing In! Kim & Kanye Go On INSANE Shopping ...  FAKE
...                                                  ...   ...
22135  Nicole Kidman Gets Honest About Her Best Beaut...  FAKE
22136  Why William, Kate Chose Willcocks for Charlott...  REAL
22137  Mamma Mia 2: Amanda Seyfried opens up on worki...  FAKE
22138  Incredibles 2 Trailer Shows Bob and Ellen Trad...  REAL
22139  Becky G Talks Guardian Angel With Tyler Henry ...  REAL

[22140 rows x 2 columns]

In [9]:
y = df.label
X_train, X_test, y_train, y_test = train_test_split(df['title'], y, test_size=0.3, random_state=53)

In [16]:
stop_words = stopwords.words('english')

In [19]:
lemmatizer=WordNetLemmatizer()
for index,row in X_train.iteritems():
    filter_sentence = ''
    
    sentence = row
    sentence = re.sub(r'[^\w\s]','',sentence) #cleaning
    
    words = nltk.word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    X_train.loc[index] = filter_sentence

In [20]:
X_train

18322     justin biebers tour cancellation i want susta...
10952     donald trump almost played president sharknado 3
8325             is tom cruise ready reunite daughter suri
21847     new year new kim kardashians secret fullbody ...
15190     the mummy london premiere canceled after manc...
                               ...                        
4596      david fosters daughter erin 35 calls his fian...
8854      katie holmes unveils new pixie cut new york city
19645     gina rodriguez wont be sacrificing her happin...
14075     prince charles demanded queen get rid favouri...
2933      7 things you didnt know about ashlee simpson ...
Name: title, Length: 15498, dtype: object

In [23]:
#Feature extraction using count vectorization and tfidf.
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X_train)
freq_term_matrix = count_vectorizer.transform(X_train)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

In [25]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in iter.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [28]:
test_counts = count_vectorizer.transform(X_test.values)
test_tfidf = tfidf.transform(test_counts)


In [31]:
test_tfidf

<6642x15886 sparse matrix of type '<class 'numpy.float64'>'
	with 68695 stored elements in Compressed Sparse Row format>

In [29]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(X_train, y_train)
pred = NB.predict(X_test)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, y_test)))
cm = confusion_matrix(y_test, pred)
cm

ValueError: could not convert string to float: ' justin biebers tour cancellation i want sustainable'